In [1]:
cd drive/My Drive/Kaggle/Painter

/content/drive/My Drive/Kaggle/Painter


In [0]:
import itertools
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import sparse_categorical_crossentropy
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import os
folders_name = os.listdir('prices')
print(len(folders_name))

742


In [0]:
train_path = 'prices'

Categories=folders_name


In [25]:
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
img_dim = 200

restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(img_dim,img_dim,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
___________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  # Remove the CWD from sys.path while we load stuff.


In [27]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=(img_dim,img_dim,3)))
model.add(Dropout(0.3))
model.add(Dense(742, activation='sigmoid'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 100352)            23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               51380736  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 742)               380646    
Total params: 75,349,094
Trainable params: 51,761,382
Non-trainable params: 23,587,712
_________________________________________________________________


In [30]:
train_batch=ImageDataGenerator(rescale=1./255).flow_from_directory(train_path, target_size=(200,200), classes=Categories, batch_size=10)

Found 5434 images belonging to 742 classes.


In [33]:
history = model.fit(train_batch, steps_per_epoch=544, epochs=1, verbose=1)

Epoch 1/1
544/544 [==============================] - 89s 163ms/step - loss: 0.1219 - accuracy: 0.9715


In [0]:
model.save('painter.h5')

In [0]:
from tensorflow.keras.models import load_model
loader = load_model('painter.h5')

In [68]:
path = 'prices/250/'
elements = os.listdir(path)
print(elements)

['Ladislas-KIJNO-LITHOGRAPHIE-FROISSEE-COLLAGE-SIGNE-1560136101.jpg', 'Jean-HOURREGUE-1560252306.jpg', 'Jean-Pierre-DASSONNEVILLE-Mont-Saint-Michel-1558884814.jpg', 'Ewa-WITKOWSKA-Mountains-1582392047.jpg', 'Wilhelm-MARTIN-See-im-Voralpenland-1589715549.jpg', 'Miela-REINA-La-donna-in-2-pezzi-1589551428.jpg', 'Erich-HARTMANN--19835--Bei-der-Mahlzeit---1589622540.jpg', 'Erich-HARTMANN--19818--Portrat-junger-Mann---1589531573.jpg', 'Erich-HARTMANN--19828--Werner-Ruwold--1589533222.jpg', 'Gustav-KOKEN-1589468598.jpg', 'Eka-PERADZE-Freedom-Sea-1589477937.jpg', 'Eka-PERADZE-Freedom-1589480614.jpg', 'Eka-PERADZE-Freedom-1589479417.jpg', 'Eka-PERADZE-Freedom-1589481724.jpg', 'Eka-PERADZE-Freedom-1589478807.jpg', 'Adolf-Gustav-DAUMILLER-1589465746.jpg', 'Eka-PERADZE-Freedom-Sea-1589477445.jpg', 'Jeff-KOONS-Bernardaud-x-Jeff-Koons-1589389911.jpg', 'Paul-VOUGA-La-Rochelle-1550776082.jpg', 'Alienor-DE-CELLES-totem-sans-tabou-1589371149.jpg', 'Alienor-DE-CELLES-Saint-Ouen-1589370272.jpg', 'Eka-PERA

In [69]:
import cv2
def prepare(filepath):
    IMG_SIZE = 200
    img_array = cv2.imread(filepath)
    img_array = img_array/255.0
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(1, IMG_SIZE, IMG_SIZE, 3)

for element in elements:
  true_path = str(path+element)
  print(true_path)
  pred1 = loader.predict([prepare(path+element)])

  print(max(pred1[0]))

  q=[]
  for i in pred1[0]:
      q.append(i)
  s = max(q)
  var3 = q.index(s)
  #print(var3)
  print(Categories[var3])
  print('')




prices/250/Ladislas-KIJNO-LITHOGRAPHIE-FROISSEE-COLLAGE-SIGNE-1560136101.jpg
0.00017960947
250

prices/250/Jean-HOURREGUE-1560252306.jpg
0.00033454577
250

prices/250/Jean-Pierre-DASSONNEVILLE-Mont-Saint-Michel-1558884814.jpg
0.000282891
250

prices/250/Ewa-WITKOWSKA-Mountains-1582392047.jpg
0.00034342177
250

prices/250/Wilhelm-MARTIN-See-im-Voralpenland-1589715549.jpg
0.00024499313
250

prices/250/Miela-REINA-La-donna-in-2-pezzi-1589551428.jpg
0.00022674158
400

prices/250/Erich-HARTMANN--19835--Bei-der-Mahlzeit---1589622540.jpg
0.00036589193
250

prices/250/Erich-HARTMANN--19818--Portrat-junger-Mann---1589531573.jpg
0.00027407552
400

prices/250/Erich-HARTMANN--19828--Werner-Ruwold--1589533222.jpg
0.00032593918
400

prices/250/Gustav-KOKEN-1589468598.jpg
0.00025149423
250

prices/250/Eka-PERADZE-Freedom-Sea-1589477937.jpg
0.000268595
400

prices/250/Eka-PERADZE-Freedom-1589480614.jpg
0.00029637897
400

prices/250/Eka-PERADZE-Freedom-1589479417.jpg
0.00016588654
400

prices/250/Eka-P